# Input Library

In [ ]:
# !pip install tensorflow==2.15.1

: 

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Input, Lambda

# Model Functional API & Sequential API

In [4]:
# seq_model = Sequential([
#     Flatten(input_shape=(28, 28)),
#     Dense(128, activation='relu'),
#     Dense(10, activation='softmax')
# ])

# TensorFlow Functional API
input = Input(shape=(28, 28))
x = Flatten()(input)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=input, outputs=predictions)

# Custom Loss Function

## Custom Huber Loss

In [5]:
def huber_loss(y_true, y_pred):
    treshold = 1
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= treshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = treshold * (tf.abs(error) - (0.5 * treshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

model.compile(loss=huber_loss, optimizer='sgd')

## Custom Huber Loss Class

In [9]:
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):
    treshold = 1
    def __init__(self, treshold):
        super().__init__()
        self.treshold = treshold
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.treshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.treshold * (tf.abs(error) - (0.5 * self.treshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    
model.compile(loss=MyHuberLoss(treshold=1), optimizer='sgd')

## Contrastive Loss

In [10]:
def contrastive_loss_with_margin(marg):
    def contrastive(y_true, y_pred):
        margin = 1
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

model.compile(loss=contrastive_loss_with_margin(marg=1), optimizer='sgd')

In [11]:
class Contrastive_Loss(Loss):
    margin = 0
    def __init__(self, margin):
        super().__init__()
        self.margin = margin
        
    def call(self, y_true, y_pred):
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(self.margin - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

# Custom Layers

## Custom Activation/Lambda Layers

In [18]:
Input_layer = Input(shape=(28, 28))
Flatten_layer = Flatten()(InputLayer)
Lambda_layer1 = Lambda(lambda x: tf.square(x))(Flatten_layer)
Dense_1 = Dense(128)(Lambda_layer1) 
Lambda_layer2 = Lambda(lambda x: tf.abs(x))(Dense_1) # Like a relu layer
Dense_2 = Dense(10, activation='softmax')(Lambda_layer2)

model = tf.keras.Model(inputs=Input_layer, outputs=Dense_2)



def my_relu(x):
    return tf.maximum(0.5, x)

model1 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    # tf.keras.layers.Dense(128, activation=my_relu),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(10)
])

## Custom Layer